In [16]:
import pandas as pd
import ast
import json

# Load json files

In [34]:
with open("../data/raw_data/2019.json", 'r') as file:
    json2019 = json.load(file)

with open("../data/raw_data/2020.json", 'r') as file:
    json2020 = json.load(file)

with open("../data/raw_data/2021.json", 'r') as file:
    json2021 = json.load(file)

with open("../data/raw_data/2022.json", 'r') as file:
    json2022 = json.load(file)

with open("../data/raw_data/2023.json", 'r') as file:
    json2023 = json.load(file)

# Extract the information

In [35]:
def get_bestsellers_info_df(data):
    books_info = []
    for item in data.values():
        results = item['results']
        bestsellers_date = results['bestsellers_date']
        for list_info in results['lists']:
            list_name = list_info['list_name_encoded']
            for book in list_info['books']:
                book_details = {
                    "title": book['title'],
                    "rank": book['rank'],
                    "publisher": book['publisher'],
                    "weeks_on_list": book['weeks_on_list'],
                    "primary_isbn13": book['primary_isbn13'],
                    "list_name": list_name,
                    "bestsellers_date": bestsellers_date
                }
                books_info.append(book_details)
    books_info_df = pd.DataFrame(books_info)
    return books_info_df

In [37]:
df_2019 = get_bestsellers_info_df(json2019)
df_2019.shape

(2490, 7)

In [38]:
df_2020 = get_bestsellers_info_df(json2020)
df_2020.shape

(2760, 7)

In [40]:
df_2021 = get_bestsellers_info_df(json2021)
df_2021.shape

(2760, 7)

In [43]:
df_2022 = get_bestsellers_info_df(json2022)
df_2022.shape

(2760, 7)

In [44]:
df_2023 = get_bestsellers_info_df(json2023)
df_2023.shape

(2760, 7)

# Create a new dataframe

In [105]:
df = pd.concat([df_2019,df_2020,df_2021,df_2022,df_2023], ignore_index=True)
df.shape

(13530, 7)

In [48]:
df.dtypes

title               object
rank                 int64
publisher           object
weeks_on_list        int64
primary_isbn13      object
list_name           object
bestsellers_date    object
dtype: object

In [46]:
df.isna().sum()

title               0
rank                0
publisher           0
weeks_on_list       0
primary_isbn13      0
list_name           0
bestsellers_date    0
dtype: int64

In [47]:
df.duplicated().sum()

0

In [49]:
df.nunique()

title               3011
rank                  15
publisher            412
weeks_on_list        545
primary_isbn13      4224
list_name             20
bestsellers_date      60
dtype: int64

# Extract the list of books and the list of lists

In [106]:
df.head(3)

,title,rank,publisher,weeks_on_list,primary_isbn13,list_name,bestsellers_date
0,THE RECKONING,1,Doubleday,9,9780385544153,combined-print-and-e-book-fiction,2018-12-22
1,EVERY BREATH,2,Grand Central,10,9781538728529,combined-print-and-e-book-fiction,2018-12-22
2,FIRE AND BLOOD,3,Bantam,5,9781524796280,combined-print-and-e-book-fiction,2018-12-22


In [111]:
title = {}
title = df['title'].apply(lambda x: x.lower()).unique()
title_df = pd.DataFrame(title,columns=['title'])
title_df

,title
0,the reckoning
1,every breath
2,fire and blood
3,where the crawdads sing
4,target: alex cross
...,...
3006,shooting iron
3007,in a holidaze
3008,collateral damage
3009,falling stars


In [110]:
# load book_info.csv
book_info = pd.read_csv('../data/database/book_info.csv')
book_titles = book_info['title'].apply(lambda x: x.lower()).unique()
book_titles_df = pd.DataFrame(book_titles,columns=['title'])
book_titles_df

,title
0,the testaments
1,normal people
2,where the forest meets the stars
3,"ask again, yes"
4,queenie
...,...
1681,misfit: growing up awkward in the '80s
1682,"unreliable narrator: me, myself, and impostor ..."
1683,america the beautiful?: one woman in a borrowe...
1684,alexandra petri's us history: important americ...


In [113]:
title_df = pd.concat([title_df,book_titles_df], ignore_index=True)
title_df['title_id'] = range(1, len(title_df) + 1)
title_df

,title,title_id
0,the reckoning,1
1,every breath,2
2,fire and blood,3
3,where the crawdads sing,4
4,target: alex cross,5
...,...,...
4692,misfit: growing up awkward in the '80s,4693
4693,"unreliable narrator: me, myself, and impostor ...",4694
4694,america the beautiful?: one woman in a borrowe...,4695
4695,alexandra petri's us history: important americ...,4696


In [116]:
title_df.to_csv('../data/database/books_title.csv', index=False)

In [117]:
lists = {}
lists = df['list_name'].unique()
lists_df = pd.DataFrame(lists,columns=['list_name'])
lists_df['list_id'] = range(1, len(lists_df) + 1)
lists_df

,list_name,list_id
0,combined-print-and-e-book-fiction,1
1,combined-print-and-e-book-nonfiction,2
2,hardcover-fiction,3
3,hardcover-nonfiction,4
4,trade-fiction-paperback,5
5,paperback-nonfiction,6
6,advice-how-to-and-miscellaneous,7
7,childrens-middle-grade-hardcover,8
8,picture-books,9
9,series-books,10


In [118]:
lists_df.to_csv('../data/database/lists_bestsellers.csv', index=False)

# Mapping the title_id and list_id

In [119]:
df['title2'] = df['title']
df['title'] = df['title2'].apply(lambda x: x.lower())

In [120]:
# Merging the DataFrames on 'title'
df_merged = pd.merge(df, title_df, on='title', how='left')
df_merged = pd.merge(df_merged, lists_df, on='list_name', how='left')

In [121]:
df_merged.columns

Index(['title', 'rank', 'publisher', 'weeks_on_list', 'primary_isbn13',
       'list_name', 'bestsellers_date', 'title2', 'title_id', 'list_id'],
      dtype='object')

In [122]:
df = df_merged[['title_id','title2','publisher','primary_isbn13','rank','weeks_on_list','list_id','bestsellers_date']]

# Dictionary mapping old column names to new ones
new_column_names = {
    'title_id': 'title_id',
    'title2': 'title',
    'publisher': 'publisher',
    'rank': 'rank',
    'weeks_on_list': 'weeks_on_list',
    'primary_isbn13': 'isbn',
    'list_id': 'list_id',
    'bestsellers_date': 'bestsellers_date'
}

# Rename the columns
df.rename(columns=new_column_names, inplace=True)

df

C:\Users\54189\AppData\Local\Temp\ipykernel_17684\1587937050.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=new_column_names, inplace=True)


,title_id,title,publisher,isbn,rank,weeks_on_list,list_id,bestsellers_date
0,1,THE RECKONING,Doubleday,9780385544153,1,9,1,2018-12-22
1,2,EVERY BREATH,Grand Central,9781538728529,2,10,1,2018-12-22
2,3,FIRE AND BLOOD,Bantam,9781524796280,3,5,1,2018-12-22
3,4,WHERE THE CRAWDADS SING,Putnam,9780735219090,4,15,1,2018-12-22
4,5,TARGET: ALEX CROSS,"Little, Brown",9780316273947,5,5,1,2018-12-22
...,...,...,...,...,...,...,...,...
16147,2962,THE HAUNTING,Delacorte,9780593481516,6,0,20,2023-11-18
16148,2904,A THOUSAND BOY KISSES,Bloom,9781728297088,7,0,20,2023-11-18
16149,2522,THE WAY I USED TO BE,Margaret K. McElderry,9781481449366,8,0,20,2023-11-18
16150,857,THE BOOK THIEF,Knopf,9780375842207,9,0,20,2023-11-18


In [124]:
# export as csv file
df.to_csv('../data/database/bestsellers.csv', index=False)